# Getting started with pyLCAIO

Welcome to the tutorial of pyLCAIO which will show you how to simply hybridize (with default values) an LCA database and an IO database. If having any problems refer to the FAQ section at the end of this notebook, if it does not answer your problem then raise an issue on Github.

PyLCAIO works with Python and will therefore require you to download Python on your machine. You can either dowload it on https://www.python.org/ or use https://www.anaconda.com/ which makes life easier.

Furthermore, the amount of RAM required for running pyLCAIO is 12GB. Below this amount of RAM, there is a high change that you will encouter a "MemoryError" during the use of pyLCAIO, for which there is no solution.

### Get the databases // necessary modules

For this tutorial, ecoinvent3.5 and exiobase3 were chosen as our LCA and IO database respectively. Therefore, you need to download the ecoinvent 3.5_cutoff_ecoSpold02.7z and ecoinvent 3.5_LCIA_implementation.7z files from https://v35.ecoquery.ecoinvent.org/File/Files and __unzip them__.  You also need to download a pxp (product by product) file of whichever reference year you want from exiobase3 monatery versions found here: https://doi.org/10.5281/zenodo.4588235

These databases in the format you just downloaded them are not readable by Python (and therefore by pyLCAIO) and must be adapted to a python-readable format. PyLCAIO relies on two external modules to do this: ecospold2matrix (https://github.com/majeau-bettez/ecospold2matrix) and pymrio (https://github.com/konstantinstadler/pymrio). You need to download these two modules.

### Create the dataframes of ecoinvent

To create the dataframe of ecoinvent, follow the tutorial on the ecospold2matrix Github page (https://github.com/majeau-bettez/ecospold2matrix/blob/master/doc/).

The code you are running with ecospold2matrix should look like this:

<img src="images/prtsc2.png">

With your own pathes to the different modules/database of course and the project name that you find suitable. Creating the dataframe will take about 30 minutes and do not panick when red lines appear, it's normal!

After using ecospold2matrix, you should have in the "out_dir" you indicated, a pickle file with a similar name to this: ecoinvent3.5.cutoffPandas_symmNorm.gz.pickle which contains the transformed ecoinvent.

Note: pickle is a storage format, just like .docx or .pdf, except it's only readable by Python.

We now have ecoinvent in dataframes, but it is in a pickle format. We need to unpickle it:

In [ ]:
import gzip
import pickle
import pandas as pd
# do not forget to change the path to your ecoinvent pickle
with gzip.open('my_path_to_the_ecoinvent_pickle','rb') as f:
    ecoinvent = pd.read_pickle(f)

Now the ecoinvent variable contains a dictionary (a collection) of dataframes for the different matrices ecoinvent provides (technosphere, biosphere, metadata, characterization).

### Create the dataframe of exiobase

Much simpler, pymrio already includes a simple parser. You just need to run the method parse_exiobase3 as follows:

In [ ]:
import sys
# still the pathes to change!
sys.path.append('my_path_to_pymrio')
import pymrio
io = pymrio.parse_exiobase3('my_path_to_exiobase')

### Run pyLCAIO

We now imported everything we needed, except pylcaio itself.

In [ ]:
import sys
# one last path to change!
sys.path.append('my_path_to_pylcaio/src/')
import pylcaio

PyLCAIO is divided into three classes: DatabaseLoader, LCAIO and Analysis.

* DatabaseLoader is the class that is assigned to successfully extract all information needed from the ecoinvent and exiobase dataframes we previously imported, and modify the information. It also loads appropriate product concordance, geography concordance, filter, STAM filter matrices and STAM categories.
* LCAIO is the class assigned to the hybridization itself. Once everything was extracted by DatabaseLoader, LCAIO incorporates this data and use it to create the hybrid database.
* Analysis is the class assigned to analyze and perform analyses on the resulting hybrid databases, such as calculation of life cycle emissions or contribution analyses. 

In object-oriented programming, every class must be initiallized to create an object of that class (here we called it database_loader). To initialize DatabaseLoader, several arguments are needed. 
- The unpickled dictionary of dataframes created through ecospold2matrix (we previously called it ecoinvent)
- The parsed exiobase file created with pymrio (we previously called it io)
- The name and version of the LCA database ('ecoinvent3.5')
- The name and version of the IO database ('exiobase3')

In [ ]:
database_loader = pylcaio.DatabaseLoader(lca_database_processed=ecoinvent,
                                         io_database_processed=io, 
                                         lca_database_name_and_version='ecoinvent3.5',
                                         io_database_name_and_version'exiobase3')

### Choose the features of your hybridization

The information required for the hybridization varies depending on which features you want the resulting hybrid database to have, said features are: endogenization of capitals of the IO database, completion of the environmental extensions of the IO database, regionalization of the impacts, use of impact world+ for results. **All these additional features are only available for the hybridization of ecoinvent3.5 and exiobase3 at present**.

Matrices for captial endogenization for Exiobase3 were compiled by Carl Sodersten (Södersten, C. J. H., Wood, R., & Hertwich, E. G. (2018). Endogenizing Capital in MRIO Models: The Implications for Consumption-Based Accounting. Environmental Science and Technology, 52(22), 13250–13259. https://doi.org/10.1021/acs.est.8b02791) and are available here:

In [ ]:
path_to_capitals = 'C://Users/Maxime/Desktop/Thesis/Bdd/EXIOBASE/Capitals/Kbar_exio_v3_6_2011pxp.mat'

Environmental extensions of Exiobase3 were completed based on the environmental extensions of USEEIO (Yang, Y., Ingwersen, W. W., Hawkins, T. R., Srocka, M., & Meyer, D. E. (2017). USEEIO: A new and transparent United States environmentally-extended input-output model. Journal of Cleaner Production, 158, 308–318. https://doi.org/10.1016/j.jclepro.2017.04.150) based on the work of Elliot Muller (Muller, E. (2019). Estimation et complétion des flux élémentaires manquants du tableau entrées-sorties multirégional EXIOBASE. Polytechnique Montreal. Retrieved from https://publications.polymtl.ca/3920/).

In [ ]:
complete_extensions = True

Impact World+ regroups the latest work in the Life Cycle Impact Assessment field (Bulle, C., Margni, M., Patouillard, L., Boulay, A. M., Bourgault, G., De Bruille, V., … Jolliet, O. (2019). IMPACT World+: a globally regionalized life cycle impact assessment method. International Journal of Life Cycle Assessment, 1653–1674. https://doi.org/10.1007/s11367-019-01583-0). Using it, will allow to compare results for the ecoinvent3.5 and exiobase3 on 15 different impact categories. Not using it restricts to 4 imapct categories.

In [ ]:
impact_world=True

Impact world+ is also the only impact assessment method that supports regionalization of impacts currently.

In [ ]:
regionalized=True

Once those features are selected, pass them as arguments to the .combine_ecoinvent_exiobase() method.

In [ ]:
lcaio_object = database_loader.combine_ecoinvent_exiobase(path_to_capitals=path_to_capitals,
                                                          complete_extensions=complete_extensions, 
                                                          impact_world=impact_world, 
                                                          regionalized=regionalized)

The lcaio_object was just initialized and ecoinvent and exiobase dataframes along with other essential parameters were transferred to it. We can now hybridize both databases with the method ".hybridize()". This method requires 2 arguments:
- The name of the method to correct double counting. There are currently two choices available: 'STAM' or 'binary'. For details on these methods and know which one to use refer to: 
    * Agez, Maxime, Guillaume Majeau-Bettez, Manuele Margni, Anders Hammer Strømman, and Réjean Samson. 2019. “Lifting the Veil on the Correction of Double Counting Incidents in Hybrid Life Cycle Assessment.” Journal of Industrial Ecology.
    * Agez, Maxime, Richard Wood, Manuele Margni, Anders Hammer Strømman, Réjean Samson, and Guillaume Majeau-Bettez. 2019. “Hybridization of Complete LCA and MRIO Databases for Comprehensive a Product System Coverage.” Journal of Industrial Ecology.
- A boolean stating if capitals will be endogenized or not

In [ ]:
lcaio_object.hybridize('STAM', True)

The operation should take around 15 to 30 minutes depending on the features selected (we are solving a 26000x26000 system here so it takes a while).

Once the calculations are ended, you can save the resulting system (all features, matrices and newly formed hybrid matrices) in a pickle format. It will avoid you having to rerun this calculation everytime.

In [ ]:
lcaio_object.save_system('pickle')

The system will be saved in the form of a hybrid_system.pickle and a description_system.txt files in your src/Databases/ folder.

**_Important_** Note that everytime you recalculate a new system (say you change the features), it will dump the newly formed system at the same place with the same name. It is thus highly recommanded to move the two files in a separate folder with a meaningful name (e.g., STAM_capitals_extended_IW_regionalized/hybrid_system.pickle).

# FAQ

__*I just saved the hybridized system and everything went smoothly. What do I do now?*__

To perform analyses on the hybridized database use the *Analysis* class of pyLCAIO. To know how to use it refer to the following notebook: *Analyze the hybrid database*

__*Error ModuleNotFound*__

To install whatever module is necessary you can use the pip install command. If the module was already installed but the code still says it cannot find that is because it is not watching at the right place. We thus need to guide it. Enter sys.path.append() and put the location of the module inside the brackets (after having put ' ' around it) and after importing the sys module.

__*Python cannot find pylcaio even though I entered the path in sys.path.append*__

Be sure to integrate src/ in your path to pylcaio.

__*I get an error in ecospold2matrix that says that it cannot find IntermediateExchanges.xml*__

You most probably forgot to unzip the ecospold file of ecoinvent.

__*I don't know what a path is and how to find it*__

A path is the adress of a file or folder on your computer. To obtain the path of an object, just check its Properties and copy paste its location. For Windows users the path should start with C://Users/, for Mac and Linux users it should start with /Users/.

__*I want to use other databases that the ones shown here*__

If you want to use different LCA/IO databases, you need to look into the DatabaseLoader module and replicate all actions for the new databases you want to hybridize together, e.g., new concordances, new filters, new STAM table, etc.

Note however, that the only available/operating LCA database is ecoinvent.GaBi, does not give detailed access to the data of their processes. The hybridization with an IO database therefore cannot happen.